# Import library

In [24]:
# #importing libraries
# import csv
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.impute import KNNImputer
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from sklearn.ensemble import RandomForestClassifier

# from boruta import BorutaPy
# from BorutaShap import BorutaShap
# from sklearn.feature_selection import RFE

# from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.combine import SMOTEENN

# from collections import Counter
# from sklearn.svm import SVC

# from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, roc_auc_score

In [25]:
# from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, roc_auc_score

# Read df

In [26]:
#this function is to read, transform and join 2 data frame

def read_features():
    path = 'input/secom.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['feature_'+str(x+1) for x in range(len(df.columns))]
    return df



def read_target():
    path = 'input/secom_labels.data'
    df = pd.read_csv(path, delimiter=' ', header=None, na_values=['NaN'])
    df.columns = ['status','timestamp']
    df['timestamp'] = pd.to_datetime(df['timestamp'],dayfirst=True)
    return df

#for the testing purporse, trim to remain first 100 rows only
X = read_features()
y = read_target().iloc[:,0]

# Remove duplicated columns

In [27]:
#find the duplicated features (columns)
def remove_duplicated_columns(df):
    dict_duplicate_pair = {}
    dict_duplicate_matches = {}
    list_duplicate = []
    to_remove = []
    for i in range(0, len(df.columns)):
        l = []
        for j in range(i+1,len(df.columns)):
            dict_duplicate_pair[str(i+1)+';'+str(j+1)] = df.iloc[:,i].equals(df.iloc[:,j])
            if df.iloc[:,i].equals(df.iloc[:,j]) == True:
                if j not in list_duplicate:
                    l.append(j)
                    to_remove.append('feature_'+str(j+1))
                list_duplicate.append(i)
                list_duplicate.append(j)
        if len(l)!=0:
            dict_duplicate_matches[i] = l


    df_duplicate_pair = pd.DataFrame.from_dict(dict_duplicate_pair, orient='index')
    df_duplicate_pair.columns=['duplicate']

    df_duplicate_matches = pd.DataFrame.from_dict(dict_duplicate_matches, orient='index')

    
    df = df.drop(columns=to_remove, axis = 1)

    return df

# X = remove_duplicated_columns(X)
# X.shape


# Remove columns with Constant volatility (std=0)

In [28]:
def remove_constant_volatility(df):
    df_EDA= df.describe().T
    df_EDA= df_EDA[df_EDA["std"] == 0]
    df = df.drop(axis=1, columns=df_EDA.index)
    return df

# X = remove_constant_volatility(X)
# X.shape

# Remove columns with high %Missing values

In [29]:
def remove_cols_with_high_pct_null(df, null_threshold):
    list_column_with_pct_null = pd.concat([df.isnull().sum(), df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
    list_column_with_pct_null= list_column_with_pct_null[list_column_with_pct_null["Percentage (%)"] >= null_threshold]
    df = df.drop(axis=1, columns=list_column_with_pct_null.index)
    return df

# X = remove_cols_with_high_pct_null(X, 0.8)
# X.shape

# Split data

In [30]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1,stratify=y)

# Outlier treatment

In [31]:
#how = ['NaN', '3s']
def replace_outlier(df, how):
    for col in df:
        ll_col = df[col].mean() - 3 * df[col].std()
        ul_col = df[col].mean() + 3 * df[col].std()
        if how == 'NaN':
            df[col] = np.where(df[col]>ul_col,np.NaN,np.where(df[col]<ll_col,np.NaN,df[col]))
        elif how == '3s':
            df[col] = np.where(df[col]>ul_col,ul_col,np.where(df[col]<ll_col,ll_col,df[col]))
    return df

# Missing value Imputation

In [32]:
#which_weights = ['distance','uniform']

def impute_null_with_knn(X_train, X_test, which_weights):
    #First scale the data 
    scaler = MinMaxScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns= X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns= X_test.columns)

    knn = KNNImputer(n_neighbors=5, weights=which_weights) #check this neighbors = 5

    X_train = pd.DataFrame(knn.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(knn.transform(X_test), columns=X_test.columns)
    
    X_train = pd.DataFrame(scaler.inverse_transform(X_train), columns= X_train.columns)
    X_test = pd.DataFrame(scaler.inverse_transform(X_test), columns= X_test.columns)
    return X_train, X_test

#X_train = impute_null_with_knn(X_train)

In [33]:
def impute_null_with_mice(X_train, X_test): 
    imp = IterativeImputer(max_iter=5, verbose=0, imputation_order='roman', random_state=0)
    X_train = pd.DataFrame(imp.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(imp.transform(X_test), columns=X_test.columns)
    return X_train, X_test

# Feature Selection

In [34]:
#list_method=['shap','gini']

def BorutaShap_FS (X, y, method_option) :
    modelshap = RandomForestClassifier(n_jobs=-1,n_estimators=100, max_depth=5, random_state=100)
    # define model for resp. classifier
    modelshap.fit(X,y)
    ##-- feature_names = np.array(X.columns)
    # define Boruta Sahp feature selection method
    feature_selector = BorutaShap(model=modelshap,
                              importance_measure=method_option,
                              classification=True)  # find all relevant features
    feature_selector.fit(X,y,n_trials=100, sample=False, verbose=False, random_state=100)  
    ##-- feature_selector.plot(which_features='accepted',figsize=(20,10))
    # call transform() on X to filter it down to selected features
    return  feature_selector.Subset()

In [35]:
#RFE function with random forest

def RFE_FS (X, y, classify) :
    feature_names = np.array(X.columns)
    if classify == 'RF':
    # define random forest classifier
        model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
        #model.fit(X, y)
        #rfe = RFE(estimator = model,n_features_to_select = 15)
    if classify== 'SVM':
        model = SVC(kernel='linear',C=5)
        #model.fit(X, y)
        #rfe = RFECV(estimator = model,scoring='accuracy')
    # find all relevant features
    model.fit(X, y)
    rfe = RFE(estimator = model,n_features_to_select = 15)
    rfe.fit(X,y)

    # check selected features
    ##-- rfe.support_

    # check ranking of features
    ##--rfe.ranking_

    # zip feature names, ranks, and decisions 
    # feature_ranks = list(zip(feature_names, 
    #                          rfe.ranking_, 
    #                          rfe.support_))

    # print the results
    ##--for feat in feature_ranks:
    ##--    print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features_rfe = list()
    indexes = np.where(rfe.ranking_ <= 1)
    for x in np.nditer(indexes):
        final_features_rfe.append(feature_names[x])
    ##-- print(final_features_rfe)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features_rfe))

In [36]:
#Boruta function with random forest

def BorutaPy_FS (X, y) :
    feature_names = np.array(X.columns)

    # define random forest classifier
    model = RandomForestClassifier(n_jobs=-1, class_weight='balanced_subsample', max_depth=5, random_state=100)
    model.fit(X, y)
    # define Boruta feature selection method
    
    feature_selector = BorutaPy(model, n_estimators='auto', verbose=0, random_state=100, max_iter=140)

    # find all relevant features
    feature_selector.fit(X.to_numpy(),y)

    # check selected features
    ##--feature_selector.support_

    # check ranking of features
    ##--feature_ranking=feature_selector.ranking_

    # zip feature names, ranks, and decisions 
    # feature_ranks = list(zip(feature_names, 
    #                          feature_selector.ranking_, 
    #                          feature_selector.support_))

    # print the results
    ##--for feat in feature_ranks:
    ##--    print('Feature: {:<30} Rank: {},  Keep: {}'.format(feat[0], feat[1], feat[2]))
        
    final_features = list()
    indexes = np.where(feature_selector.ranking_ <= 1)
    for x in np.nditer(indexes):
        final_features.append(feature_names[x])
    ##--print(final_features)
    
 # call transform() on X to filter it down to selected features
    return pd.DataFrame(X.filter(final_features))

# Balancing

In [37]:
def sampling(X_train, y_train, sampler):
    
    #SMOTE
    if sampler == 'SMOTE':
        sampler = SMOTE(random_state=100)    
    
    #ROSE
    if sampler == 'ROSE':
        sampler = RandomOverSampler(random_state=100, shrinkage=1)

    #ADASYN
    if sampler == 'ADASYN':
        sampler = ADASYN(random_state=100)
    

    #SMOTTEENN
    if sampler == 'SMOTEENN' :
        sampler = SMOTEENN(random_state=100)
        
        
    #Random under Sampling
    if sampler == "randomunder":
        sampler = RandomUnderSampler(random_state=100)

    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    counter = Counter(y_resampled)
    print(counter)
    
    return X_resampled, y_resampled

# X_train, y_train = sampling(X_train, y_train,'SMOTE')
# X_train.shape

# Model

In [38]:
def run_model(X_train, y_train, X_test, y_test):
    # building model before balancing data
    model = RandomForestClassifier(random_state=1, n_estimators=1000, max_depth=5)
    model.fit(X_train,y_train)


    #For TEST SPLIT
    y_pred= model.predict(X_test)
    cf_matrix = confusion_matrix(y_test, y_pred)
    accuracy= accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred) ##
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    specificity = cf_matrix[1][1] / ( cf_matrix[1][1] + cf_matrix[1][0] )
    auc = roc_auc_score(y_test, y_pred)
    type_1_error_FP = cf_matrix[0][1]
    type_2_error_FN = cf_matrix[1][0]
    #Note by default 1 is the positive label. Therefore, -1 is negative

    #For TRAIN SPLIT
    y_pred_train= model.predict(X_train)
    cf_matrix_train = confusion_matrix(y_train, y_pred_train)
    accuracy_train= accuracy_score(y_train, y_pred_train)
    f1_train = f1_score(y_train, y_pred_train) ##
    precision_train = precision_score(y_train, y_pred_train)
    recall_train = recall_score(y_train, y_pred_train)
    specificity_train = cf_matrix_train[1][1] / ( cf_matrix_train[1][1] + cf_matrix_train[1][0] )
    auc_train = roc_auc_score(y_train, y_pred_train)
    type_1_error_FP_train = cf_matrix_train[0][1]
    type_2_error_FN_train = cf_matrix_train[1][0]


    return cf_matrix, accuracy, f1, precision, recall, specificity, type_1_error_FP, type_2_error_FN, auc, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, specificity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train

#run_model(X_train, y_train, X_test, y_test)

## Combine

In [22]:
fast_track_cols = X_test.columns

In [46]:
df_result = pd.DataFrame(result, columns = ['No.','combination','cols','cf_matrix', 'accuracy', 'f1', 'precision', 'recall', 'specificity', 'type_1_error_FP', 'type_2_error_FN', 'auc', 'cf_matrix_train', 'accuracy_train', 'f1_train', 'precision_train', 'recall_train', 'specificity_train', 'type_1_error_FP_train', 'type_2_error_FN_train', 'auc_train'])
df_result.to_csv('output/result.csv')
df_result

,No.,combination,cols,cf_matrix,accuracy,f1,precision,recall,specificity,type_1_error_FP,...,auc,cf_matrix_train,accuracy_train,f1_train,precision_train,recall_train,specificity_train,type_1_error_FP_train,type_2_error_FN_train,auc_train
0,0,3s&knn__distance&BoP&SMOTE,"Index(['feature_20', 'feature_22', 'feature_34...","[[238, 55], [12, 9]]",0.786624,0.211765,0.140625,0.428571,0.428571,55,...,0.620429,"[[953, 217], [96, 1074]]",0.866239,0.872816,0.831913,0.917949,0.917949,217,96,0.866239
1,1,3s&knn__distance&BoS__shap&SMOTE,"Index(['feature_34', 'feature_60', 'feature_47...","[[228, 65], [6, 15]]",0.773885,0.297030,0.187500,0.714286,0.714286,65,...,0.746221,"[[921, 249], [185, 985]]",0.814530,0.819468,0.798217,0.841880,0.841880,249,185,0.814530
2,2,3s&knn__distance&RFE__RF&SMOTE,"Index(['feature_1', 'feature_20', 'feature_22'...","[[248, 45], [13, 8]]",0.815287,0.216216,0.150943,0.380952,0.380952,45,...,0.613684,"[[1022, 148], [37, 1133]]",0.920940,0.924521,0.884465,0.968376,0.968376,148,37,0.920940
3,3,3s&MICE&BoP&SMOTE,"Index(['feature_1', 'feature_20', 'feature_22'...","[[233, 60], [9, 12]]",0.780255,0.258065,0.166667,0.571429,0.571429,60,...,0.683325,"[[958, 212], [35, 1135]]",0.894444,0.901867,0.842613,0.970085,0.970085,212,35,0.894444


In [47]:
X = read_features()
y = read_target().iloc[:,0]

result = []
i = 0
f = open('output/tracker.csv', 'w')
# create the csv writer
writer = csv.writer(f)



#step 1:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify=y)


#-----------
# #step 2:
# X_train = remove_duplicated_columns(X_train)

# #step 3:
# X_train = remove_constant_volatility(X_train)
# #step 4:
# X_train = remove_cols_with_high_pct_null(X_train, 0.8) #this can be in the loop too, may be later

# #step 5: remove the same columns from step 2-3 TRAIN_TEST split
# X_test = X_test.loc[:,X_train.columns]

#------------


X_train = X_train.loc[:, fast_track_cols]
X_test = X_test.loc[:, fast_track_cols]

#------------


#step 6-9
replace_outlier_options = ['3s']
#replace_outlier_options = ['3s','NaN']
impute_null_options = ['knn__distance', 'MICE']
#impute_null_options = ['knn__distance', 'MICE', 'knn__uniform']
FS_options = ['BoP','BoS__shap', 'RFE__RF' ]
#FS_options = ['BoP','BoS__shap', 'BoS__gini', 'RFE__RF', 'RFE__SVM' ]
sampling_options = ['SMOTE']
#sampling_options = ['SMOTE','ROSE','ADASYN','SMOTEENN']

for replace_with in replace_outlier_options:
    for knn_weight in impute_null_options:
        for classifier_model in FS_options:
            #<remove correlated columns, decide on the thresold 70%>
            for sampling_technique in sampling_options:
                X_train_temp = X_train
                X_test_temp = X_test
                y_train_temp = y_train
                y_test_temp = y_test

                #step 6: oulier treatement (on both TRAIN & TEST split)
                X_train_temp = replace_outlier(X_train_temp, replace_with)
                X_test_temp = replace_outlier(X_test_temp, replace_with)
                
                #step 7: missing value imputation (on both TRAIN & TEST split)
                if knn_weight == 'knn__distance' or knn_weight == 'knn__uniform':
                    X_train_temp, X_test_temp = impute_null_with_knn(X_train_temp, X_test_temp, knn_weight[-(len(knn_weight)-5):])
                elif knn_weight == 'MICE':
                    X_train_temp, X_test_temp = impute_null_with_mice(X_train_temp, X_test_temp)

                #step 8: feature selection (on both TRAIN & TEST split)
                if classifier_model == 'BoS__shap' or classifier_model == 'BoS__gini':
                    X_train_temp = BorutaShap_FS(X_train_temp, y_train_temp, classifier_model[-(len(classifier_model)-5):])
                elif classifier_model == 'RFE__RF' or classifier_model == 'RFE__SVM':
                    X_train_temp = RFE_FS(X_train_temp, y_train_temp, classifier_model[-(len(classifier_model)-5):])
                elif classifier_model == 'BoP':
                    X_train_temp = BorutaPy_FS(X_train_temp, y_train_temp)
                
                #apply the same result for TEST
                X_test_temp = X_test_temp.loc[:,X_train_temp.columns]


                #step 9: balancing only on TRAIN split
                X_train_temp, y_train_temp = sampling(X_train_temp, y_train_temp, sampling_technique)

                #print out datasets for backup
                X_train_temp.to_csv('output/X_train_temp_'+str(i)+'.csv')
                X_test_temp.to_csv('output/X_test_temp_'+str(i)+'.csv')
                y_train_temp.to_csv('output/y_train_temp_'+str(i)+'.csv')
                y_test_temp.to_csv('output/y_test_temp_'+str(i)+'.csv')
                


                #step 10: train model, predict, and print scores
                cf_matrix, accuracy, f1, precision, recall, specificity, type_1_error_FP, type_2_error_FN, auc, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, specificity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train = run_model(X_train_temp, y_train_temp, X_test_temp, y_test_temp)

                combined_technique = replace_with +'&'+ knn_weight +'&'+ classifier_model +'&'+ sampling_technique
                result.append((i, combined_technique, X_train_temp.columns, cf_matrix, accuracy, f1, precision, recall, specificity, type_1_error_FP, type_2_error_FN, auc, cf_matrix_train, accuracy_train, f1_train, precision_train, recall_train, specificity_train, type_1_error_FP_train, type_2_error_FN_train, auc_train))
                
                print(i, combined_technique,'\n acc', accuracy,'\n accuracy_train',accuracy_train, '\n f1', f1,'\n f1_train',f1_train, '\n spec', specificity, '\n auc', auc, '\n cfm', '\n', cf_matrix, '\n cols', X_train_temp.columns, '\n')
                #print(i, combined_technique)
                row = str(i) +' '+ combined_technique +' acc '+ str(accuracy) +' f1 '+ str(f1) +' acc_train '+ str(accuracy_train) +' f1_train '+ str(f1_train)
                writer.writerow(row)

                i+=1



df_result = pd.DataFrame(result, columns = ['No.','combination','cols','cf_matrix', 'accuracy', 'f1', 'precision', 'recall', 'specificity', 'type_1_error_FP', 'type_2_error_FN', 'auc', 'cf_matrix_train', 'accuracy_train', 'f1_train', 'precision_train', 'recall_train', 'specificity_train', 'type_1_error_FP_train', 'type_2_error_FN_train', 'auc_train'])
df_result.to_csv('output/result.csv')
print(result)
# close the file
f.close()

Counter({-1: 1170, 1: 1170})
0 3s&knn__distance&BoP&SMOTE 
 acc 0.7866242038216561 
 accuracy_train 0.8662393162393163 
 f1 0.2117647058823529 
 f1_train 0.8728159284843559 
 spec 0.42857142857142855 
 auc 0.6204290589956118 
 cfm 
 [[238  55]
 [ 12   9]] 
 cols Index(['feature_20', 'feature_22', 'feature_34', 'feature_60', 'feature_122',
       'feature_130', 'feature_131', 'feature_206', 'feature_248',
       'feature_342', 'feature_478', 'feature_520'],
      dtype='object') 



  0%|          | 0/100 [00:00<?, ?it/s]

In [69]:
lst =[]
a=1
b=2
lst.append((a,b))
lst.append((b,a))

df = pd.DataFrame(lst,
               columns =['Name', 'val'])
df

,Name,val
0,1,2
1,2,1


In [66]:
# list of strings
lst = ['Geeks', 'For', 'Geeks', 'is', 'portal', 'for', 'Geeks']
  
# list of int
lst2 = [11, 22, 33, 44, 55, 66, 77]
  
# Calling DataFrame constructor after zipping
# both lists, with columns specified
df = pd.DataFrame(list(zip(lst, lst2)),
               columns =['Name', 'val'])
df

,Name,val
0,Geeks,11
1,For,22
2,Geeks,33
3,is,44
4,portal,55
5,for,66
6,Geeks,77


# --------------------Appendix--------------------

In [ ]:
# #------------------------------------
# X_train = replace_outlier(X_train, how)
# replace_outlier_options = ['NaN', '3s']
# #--
# X_train = impute_null_with_knn(X_train, which_weights) #
# impute_null_options = ['knn__distance', 'knn__uniform', 'MICE']

# X_train = impute_null_with_mice(X_train)
# #--
# BorutaShap_FS(X_train, y_train, method_option)
# list_method=['shap','gini']

# RFE_FS(X_train, y_train, classify) 
# list_clf=['RF','SVM']

# BorutaPy_FS(X_train, y_train)

# FS_options = ['BoS__shap', 'BoS__gini', 'RFE__RF', 'RFE__SVM', 'BoP']
# #--
# sampling(X_train, y_train, sampler)
# sampling_options = ['SMOTE','ROSE','ADASYN','SMOTEENN']

# #--------------------------------------

In [9]:
# l = []
# a=1
# b=2
# c=3

# l.append([a,b,c])
# l.append([b,c,a])

# l



[[1, 2, 3], [2, 3, 1]]

In [ ]:
# list_null_impute = [knnimputation_distance, MICEimputation_distance]
# list_null_outlier = [outlier_knn,outlier_3s]
# list_feat_selection = ['Boruta_RF', 'Boruta_shap', 'RFE']
# Boruta = ['RF', 'XGB']
# Boruta_shap = ['RF', 'XGB', 'kNN']
# RFE = ['RF', 'SVC']

In [ ]:
# #TESTING
# df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [1,1,1]]),
#                    columns=['a', 'b', 'c'])
# df

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9
3,1,1,1
